In [1]:
# %%
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Assuming `model` is your trained model


# %%
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
data['short_pass'] = (data['air_yards'] < 10).astype(int)
data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
data['deep_pass'] = (data['air_yards'] > 19).astype(int)
data['end_zone_target'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['complete_pass'] * ppr +          # 1 point per completion
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


C:\Users\src15\AppData\Local\Temp\ipykernel_19252\237515004.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\237515004.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\237515004.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [2]:
data['season'].value_counts()


season
2021    45049
2023    44877
2022    44558
2019    43010
2020    43004
2018    42697
2024    17620
Name: count, dtype: int64

In [3]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [4]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
]

    


C:\Users\src15\AppData\Local\Temp\ipykernel_19252\1090877097.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\1090877097.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\1090877097.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [5]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


# Fantasy Points Model

In [6]:


# Load your data
# data = pd.read_csv('your_data.csv')

# Prepare the features (X) and target (y)
predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location','season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]
X = df[predictors]
y = df['fantasy_points']

# Convert categorical variables (if needed)
X = pd.get_dummies(X, columns=['pass_location'], drop_first=True)

# Handle missing values if any
X.fillna(X.mean(), inplace=True)  # Example for filling with mean

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Make predictions with the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

# Feature importance
importance = best_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

# Print the feature importance
print("Feature Importance:")
print(feature_importance_df)


Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Mean Squared Error: 3.79
R^2 Score: 0.07
Feature Importance:
                       Feature  Importance
7  distance_to_EZ_after_target    0.396511
5                       qb_hit    0.139016
1                 yardline_100    0.089789
8         pass_location_middle    0.073451
0                    air_yards    0.071956
6              end_zone_target    0.066657
3                         down    0.054076
2                      ydstogo    0.041806
4                       season    0.033997
9          pass_location_right    0.032741


In [7]:
receivers = throws[[
    'air_yards', 'yardline_100', 'ydstogo', 'implied_posteam_total',
    'down', 'pass_location', 'season', 'qb_hit', 'end_zone_target','distance_to_EZ_after_target']]

In [8]:
throws1 = pd.get_dummies(throws, columns=['pass_location'], drop_first=True)


In [9]:
receivers = throws1[[
    'air_yards', 'yardline_100', 'ydstogo', 
    'down', 'season', 'qb_hit', 'end_zone_target','distance_to_EZ_after_target','pass_location_middle','pass_location_right']]

In [10]:
throws['xFPs'] = best_model.predict(receivers)

C:\Users\src15\AppData\Local\Temp\ipykernel_19252\3586448896.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  throws['xFPs'] = best_model.predict(receivers)


In [11]:
#throws[throws['xFPs']>4][['desc','week','yardline_100','air_yards','end_zone_target','receiver_player_name','fantasy_points','xFPs','yards_gained','touchdown','implied_posteam_total','pass_location','distance_to_EZ_after_target']].sort_values('xFPs',ascending=False).head(50)

In [12]:
#throws[throws['air_yards']>50][['desc','yardline_100','air_yards','end_zone_target','receiver_player_name','fantasy_points','xFPs','yards_gained','touchdown','pass_location','qb_hit','down','distance_to_EZ_after_target']]

In [13]:
#throws[[
#    'air_yards', 'yardline_100', 'ydstogo', 'implied_posteam_total',
#    'down', 'week', 'season', 'qb_hit', 'end_zone_target','xFPs','fantasy_points','distance_to_EZ_after_target']].corr().sort_values('xFPs',ascending=False)

In [14]:
receiver_szns = throws.groupby(['receiver_player_name','posteam','season']).agg({'xFPs':'sum','fantasy_points':'sum','game_id':'nunique','pass':'sum','air_yards':'sum','end_zone_target':'sum','touchdown':'sum'})

receiver_szns['Diff'] = receiver_szns['fantasy_points'] - receiver_szns['xFPs']

In [15]:
receiver_szns.corr()

,xFPs,fantasy_points,game_id,pass,air_yards,end_zone_target,touchdown,Diff
xFPs,1.000000,0.981444,0.801578,0.995293,0.904300,0.828758,0.832525,0.129511
fantasy_points,0.981444,1.000000,0.782725,0.977593,0.875272,0.807910,0.889590,0.317240
game_id,0.801578,0.782725,1.000000,0.818013,0.636335,0.608052,0.640306,0.083232
pass,0.995293,0.977593,0.818013,1.000000,0.869375,0.785447,0.808012,0.132874
air_yards,0.904300,0.875272,0.636335,0.869375,1.000000,0.863748,0.777350,0.053776
end_zone_target,0.828758,0.807910,0.608052,0.785447,0.863748,1.000000,0.814612,0.079047
touchdown,0.832525,0.889590,0.640306,0.808012,0.777350,0.814612,1.000000,0.482806
Diff,0.129511,0.317240,0.083232,0.132874,0.053776,0.079047,0.482806,1.000000


In [16]:
#receiver_szns.sort_values('Diff',ascending=True).head(50)

## 2024 Results

In [17]:
new_data = data[data['season']==2024]



    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
new_data = new_data[new_data['air_yards'].notna()]
new_data = new_data[new_data['receiver_player_name'].notna()]

In [18]:
# Prepare features from the new dataset
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'week', 'season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

new_X = new_data[new_predictors]

In [19]:
new_X = pd.get_dummies(new_X, columns=['pass_location'], drop_first=True)


missing_cols = set(X.columns) - set(new_X.columns)
for col in missing_cols:
    new_X[col] = 0  # Fill with zero or any other appropriate value
new_X = new_X[X.columns]  # Reorder columns to match training set

In [20]:
new_X

,air_yards,yardline_100,ydstogo,down,season,qb_hit,end_zone_target,distance_to_EZ_after_target,pass_location_middle,pass_location_right
263197,-3.0,67.0,7,2.0,2024,0.0,False,70.0,False,False
263198,2.0,45.0,10,1.0,2024,0.0,False,43.0,True,False
263202,6.0,30.0,6,3.0,2024,0.0,False,24.0,True,False
263203,12.0,22.0,10,1.0,2024,0.0,False,10.0,False,True
263208,5.0,5.0,5,3.0,2024,0.0,True,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...
280796,3.0,70.0,10,1.0,2024,0.0,False,67.0,False,True
280799,26.0,56.0,21,3.0,2024,0.0,False,30.0,False,False
280803,18.0,80.0,9,3.0,2024,0.0,False,62.0,False,False
280806,-2.0,55.0,12,3.0,2024,0.0,False,57.0,False,False


In [21]:
predictions = best_model.predict(new_X)


In [22]:





# Make predictions using the trained model
predictions = best_model.predict(new_X)

# Add predictions to the new dataset (optional)
new_data['xFPs'] = predictions

# Save or display the new dataset with predictions
# new_data.to_csv('new_data_with_predictions.csv', index=False)


C:\Users\src15\AppData\Local\Temp\ipykernel_19252\1872298771.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data['xFPs'] = predictions


In [23]:
new_data[['desc','air_yards']]

,desc,air_yards
263197,(14:27) 1-K.Murray pass short left to 6-J.Conn...,-3.0
263198,(13:43) (Shotgun) 1-K.Murray pass short middle...,2.0
263202,(11:08) (Shotgun) 1-K.Murray pass short middle...,6.0
263203,(10:32) (Shotgun) 1-K.Murray pass incomplete s...,12.0
263208,(7:54) (Shotgun) 1-K.Murray pass short left to...,5.0
...,...,...
280796,(6:17) 14-S.Darnold pass short right to 84-J.O...,3.0
280799,(4:24) (Shotgun) 14-S.Darnold pass incomplete ...,26.0
280803,(3:26) (Shotgun) 9-M.Stafford pass deep left t...,18.0
280806,(2:00) (Shotgun) 9-M.Stafford pass short left ...,-2.0


In [24]:
this_szn = new_data.groupby(['receiver_player_name','posteam']).agg({'fantasy_points':'sum','xFPs':'sum','pass':'sum','air_yards':'mean','end_zone_target':'sum','distance_to_EZ_after_target':'mean'}).sort_values('xFPs',ascending=False)

In [25]:
50+54.4+24

128.4

In [26]:
this_szn.corr()

,fantasy_points,xFPs,pass,air_yards,end_zone_target,distance_to_EZ_after_target
fantasy_points,1.000000,0.949290,0.946414,0.258470,0.670869,-0.084767
xFPs,0.949290,1.000000,0.991538,0.316809,0.734145,-0.104968
pass,0.946414,0.991538,1.000000,0.258544,0.664939,-0.052522
air_yards,0.258470,0.316809,0.258544,1.000000,0.387257,-0.488125
end_zone_target,0.670869,0.734145,0.664939,0.387257,1.000000,-0.282665
distance_to_EZ_after_target,-0.084767,-0.104968,-0.052522,-0.488125,-0.282665,1.000000


In [27]:
this_szn['Diff'] = this_szn['fantasy_points'] - this_szn['xFPs']
this_szn.sort_values('xFPs',ascending=False).head(5)

,,fantasy_points,xFPs,pass,air_yards,end_zone_target,distance_to_EZ_after_target,Diff
receiver_player_name,posteam,,,,,,,
G.Wilson,NYJ,116.0,132.848099,75,8.200000,4,40.040000,-16.848099
D.London,ATL,123.1,113.377953,62,10.177419,6,34.887097,9.722047
D.Metcalf,SEA,109.8,110.737526,61,13.622951,5,42.196721,-0.937526
J.Jefferson,MIN,135.6,109.249748,60,12.450000,6,40.166667,26.350252
M.Nabers,NYG,99.7,106.475266,60,10.750000,4,38.716667,-6.775266


In [28]:
this_szn.loc['D.Smith', 'PHI']

fantasy_points                 67.100000
xFPs                           59.100555
pass                           34.000000
air_yards                       9.764706
end_zone_target                 3.000000
distance_to_EZ_after_target    43.676471
Diff                            7.999445
Name: (D.Smith, PHI), dtype: float64

# Yardage Model

In [29]:


# Load your data
# data = pd.read_csv('your_data.csv')

# Prepare the features (X) and target (y)
predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]
X = df[predictors]
y = df['yards_gained']

# Convert categorical variables (if needed)
X = pd.get_dummies(X, columns=['pass_location'], drop_first=True)

# Handle missing values if any
X.fillna(X.mean(), inplace=True)  # Example for filling with mean

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
yardage_model = grid_search.best_estimator_

# Make predictions with the best model
y_pred = yardage_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

# Feature importance
importance = yardage_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

# Print the feature importance
print("Feature Importance:")
print(feature_importance_df)


Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Mean Squared Error: 85.14
R^2 Score: 0.12
Feature Importance:
                       Feature  Importance
0                    air_yards    0.301195
1                 yardline_100    0.205585
6              end_zone_target    0.136583
7  distance_to_EZ_after_target    0.099941
5                       qb_hit    0.085188
8         pass_location_middle    0.076112
3                         down    0.043744
9          pass_location_right    0.022422
2                      ydstogo    0.018170
4                       season    0.011060


In [72]:
with open('yardage_model.pkl', 'wb') as file:
    pickle.dump(yardage_model, file)

# Touchdown Model

In [30]:


# Load your data
# data = pd.read_csv('your_data.csv')

# Prepare the features (X) and target (y)
predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]
X = df[predictors]
y = df['pass_touchdown']

# Convert categorical variables (if needed)
X = pd.get_dummies(X, columns=['pass_location'], drop_first=True)

# Handle missing values if any
X.fillna(X.mean(), inplace=True)  # Example for filling with mean

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
touchdown_model = grid_search.best_estimator_

# Make predictions with the best model
y_pred = touchdown_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

# Feature importance
importance = touchdown_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

# Print the feature importance
print("Feature Importance:")
print(feature_importance_df)


Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Mean Squared Error: 0.03
R^2 Score: 0.26
Feature Importance:
                       Feature  Importance
7  distance_to_EZ_after_target    0.599105
6              end_zone_target    0.210732
1                 yardline_100    0.083220
0                    air_yards    0.026237
2                      ydstogo    0.021442
5                       qb_hit    0.019292
8         pass_location_middle    0.012701
9          pass_location_right    0.012111
3                         down    0.008291
4                       season    0.006870


In [73]:
with open('touchdown_model.pkl', 'wb') as file:
    pickle.dump(touchdown_model, file)

# Comparing Component-Based xFP to General xFP

In [31]:
new_X = new_data[new_predictors]

# Convert categorical variables (if needed)
new_X = pd.get_dummies(new_X, columns=['pass_location'], drop_first=True)

# Handle missing values (example: fill with mean)
#new_X.fillna(new_X.mean(), inplace=True)

# Ensure the new dataset has the same feature columns as the training set
# If the training set had features that were removed or had additional categories,
# align the new data with the training features
missing_cols = set(X.columns) - set(new_X.columns)
for col in missing_cols:
    new_X[col] = 0  # Fill with zero or any other appropriate value
new_X = new_X[X.columns]  # Reorder columns to match training set

# Make predictions using the trained model
x_yards = yardage_model.predict(new_X)

xTDs = touchdown_model.predict(new_X)


# Add predictions to the new dataset (optional)
new_data['xYards'] = x_yards
new_data['xTDs'] = xTDs

C:\Users\src15\AppData\Local\Temp\ipykernel_19252\582892723.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data['xYards'] = x_yards
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\582892723.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data['xTDs'] = xTDs


In [32]:
new_data['composite_xFPs'] = new_data['cp'] * ppr + new_data['xYards']*0.1 + new_data['xTDs']*6

C:\Users\src15\AppData\Local\Temp\ipykernel_19252\869558254.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data['composite_xFPs'] = new_data['cp'] * ppr + new_data['xYards']*0.1 + new_data['xTDs']*6


In [33]:
new_data[['fantasy_points','xFPs','composite_xFPs']].corr()

,fantasy_points,xFPs,composite_xFPs
fantasy_points,1.000000,0.238913,0.245785
xFPs,0.238913,1.000000,0.976732
composite_xFPs,0.245785,0.976732,1.000000


# Season-over-season Comparison

In [34]:
df['season'].value_counts()

season
2021    18055
2023    17483
2020    17307
2022    17305
2018    17172
2019    17140
Name: count, dtype: int64

In [35]:
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

test_df = df[new_predictors]

test_df.head()

,air_yards,yardline_100,ydstogo,down,pass_location,season,qb_hit,end_zone_target,distance_to_EZ_after_target
2,8.0,80.0,15,1.0,right,2018,0.0,False,72.0
5,4.0,39.0,10,1.0,right,2018,0.0,False,35.0
6,-3.0,39.0,10,2.0,left,2018,0.0,False,42.0
7,24.0,39.0,10,3.0,left,2018,0.0,False,15.0
10,1.0,1.0,1,3.0,right,2018,0.0,True,0.0


In [36]:
test_df = pd.get_dummies(test_df, columns=['pass_location'], drop_first=True)


# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict(test_df)
df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['cp'] * ppr + df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_19252\2134994946.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\2134994946.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\2134994946.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [37]:
df[['xFPs','compositeXFP','fantasy_points']].corr()

,xFPs,compositeXFP,fantasy_points
xFPs,1.000000,0.981968,0.290096
compositeXFP,0.981968,1.000000,0.279971
fantasy_points,0.290096,0.279971,1.000000


In [38]:
receivers = df.groupby(['receiver_player_id','posteam','season']).agg({'receiver_player_name':'max','pass':'sum','xTDs':'sum','pass_touchdown':'sum','xYards':'sum','yards_gained':'sum','cp':'sum','complete_pass':'sum','xFPs':'sum','fantasy_points':'sum','compositeXFP':'sum'}).sort_values('xFPs',ascending=False)

In [39]:
receivers

,,,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,xFPs,fantasy_points,compositeXFP
receiver_player_id,posteam,season,,,,,,,,,,,
00-0033908,LA,2021,C.Kupp,191,11.644811,16.0,1441.202759,1947.0,129.081419,145.0,342.066803,441.7,343.070562
00-0036358,DAL,2023,C.Lamb,181,11.840400,12.0,1385.905273,1749.0,117.402901,135.0,328.151093,381.9,327.035831
00-0036322,MIN,2022,J.Jefferson,184,10.633155,8.0,1452.039917,1819.0,116.943704,128.0,327.215942,357.9,325.946621
00-0031381,LV,2022,D.Adams,180,10.218541,14.0,1525.546143,1516.0,112.967685,100.0,325.018219,341.6,326.833543
00-0032765,NO,2019,M.Thomas,185,7.326191,9.0,1457.892822,1725.0,128.447433,149.0,322.272430,375.5,318.193874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
00-0036355,LAC,2023,J.Herbert,1,0.004950,0.0,1.631526,10.0,0.748702,1.0,0.739431,2.0,0.941556
00-0025415,SF,2018,J.Staley,1,0.005922,0.0,1.069185,-5.0,0.670360,1.0,0.736195,0.5,0.812811
00-0029263,SEA,2018,R.Wilson,1,0.006025,0.0,0.510591,-11.0,0.570121,1.0,0.683728,-0.1,0.657332


In [40]:
receivers_qual = receivers[receivers['pass']>=20]

In [41]:
receivers

,,,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,xFPs,fantasy_points,compositeXFP
receiver_player_id,posteam,season,,,,,,,,,,,
00-0033908,LA,2021,C.Kupp,191,11.644811,16.0,1441.202759,1947.0,129.081419,145.0,342.066803,441.7,343.070562
00-0036358,DAL,2023,C.Lamb,181,11.840400,12.0,1385.905273,1749.0,117.402901,135.0,328.151093,381.9,327.035831
00-0036322,MIN,2022,J.Jefferson,184,10.633155,8.0,1452.039917,1819.0,116.943704,128.0,327.215942,357.9,325.946621
00-0031381,LV,2022,D.Adams,180,10.218541,14.0,1525.546143,1516.0,112.967685,100.0,325.018219,341.6,326.833543
00-0032765,NO,2019,M.Thomas,185,7.326191,9.0,1457.892822,1725.0,128.447433,149.0,322.272430,375.5,318.193874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
00-0036355,LAC,2023,J.Herbert,1,0.004950,0.0,1.631526,10.0,0.748702,1.0,0.739431,2.0,0.941556
00-0025415,SF,2018,J.Staley,1,0.005922,0.0,1.069185,-5.0,0.670360,1.0,0.736195,0.5,0.812811
00-0029263,SEA,2018,R.Wilson,1,0.006025,0.0,0.510591,-11.0,0.570121,1.0,0.683728,-0.1,0.657332


In [42]:
receivers_qual.reset_index().sort_values(['receiver_player_id', 'season'])

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,xFPs,fantasy_points,compositeXFP
794,00-0021547,LAC,2018,A.Gates,45,3.830350,2.0,310.046295,333.0,27.173962,28.0,82.315056,79.3,81.160690
396,00-0022127,DAL,2019,J.Witten,83,3.527425,4.0,570.964417,529.0,57.494382,63.0,137.251511,139.9,135.755377
152,00-0022921,ARI,2018,L.Fitzgerald,112,6.115843,6.0,880.963379,734.0,72.544649,69.0,200.287949,178.4,197.336049
174,00-0022921,ARI,2019,L.Fitzgerald,107,6.291249,4.0,798.765869,835.0,73.160045,73.0,192.898148,192.5,190.784130
491,00-0022921,ARI,2020,L.Fitzgerald,72,2.179992,1.0,524.440125,409.0,52.608162,54.0,118.390869,100.9,118.132127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,00-0039075,LA,2023,P.Nacua,160,6.517516,6.0,1289.263794,1486.0,108.953619,105.0,277.112396,289.6,276.985090
592,00-0039139,DET,2023,J.Gibbs,71,1.703329,1.0,381.663513,316.0,56.332290,52.0,105.348976,95.6,104.718618
866,00-0039144,GB,2023,L.Musgrave,46,1.336321,1.0,345.465942,352.0,31.674964,34.0,75.258263,75.2,74.239485
256,00-0039146,GB,2023,J.Reed,94,5.873113,8.0,734.636780,793.0,59.541225,64.0,168.796036,191.3,168.243584


In [43]:
df = receivers_qual.reset_index().sort_values(['receiver_player_id', 'season'])

# Create lagged columns for the next season's stats
for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass', 'xFPs', 'fantasy_points', 'compositeXFP']:
    df[f'{col}_next'] = df.groupby('receiver_player_id')[col].shift(-1)

# Drop rows where the next season's data is missing (last season for each player)
df_lagged = df.dropna(subset=[f'{col}_next' for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass', 'xFPs', 'fantasy_points', 'compositeXFP']])

In [44]:
df.head(20)

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,xFPs_next,fantasy_points_next,compositeXFP_next
794,00-0021547,LAC,2018,A.Gates,45,3.830350,2.0,310.046295,333.0,27.173962,...,81.160690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,00-0022127,DAL,2019,J.Witten,83,3.527425,4.0,570.964417,529.0,57.494382,...,135.755377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,00-0022921,ARI,2018,L.Fitzgerald,112,6.115843,6.0,880.963379,734.0,72.544649,...,197.336049,6.291249,4.0,798.765869,835.0,73.160045,73.0,192.898148,192.5,190.784130
174,00-0022921,ARI,2019,L.Fitzgerald,107,6.291249,4.0,798.765869,835.0,73.160045,...,190.784130,2.179992,1.0,524.440125,409.0,52.608162,54.0,118.390869,100.9,118.132127
491,00-0022921,ARI,2020,L.Fitzgerald,72,2.179992,1.0,524.440125,409.0,52.608162,...,118.132127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,00-0022943,NO,2018,B.Watson,46,4.272406,2.0,365.639679,400.0,29.137302,...,91.335703,1.391622,0.0,176.140900,173.0,15.453680,17.0,42.463737,34.3,41.417503
1316,00-0022943,NE,2019,B.Watson,24,1.391622,0.0,176.140900,173.0,15.453680,...,41.417503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1398,00-0023564,PHI,2018,D.Sproles,23,1.336910,2.0,124.240898,160.0,17.396862,...,37.842414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,00-0024221,WAS,2018,V.Davis,36,0.959229,2.0,312.337250,367.0,22.395950,...,59.385049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,00-0024243,GB,2021,M.Lewis,28,2.106592,0.0,160.565704,214.0,19.881775,...,48.577900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_lagged

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,xFPs_next,fantasy_points_next,compositeXFP_next
152,00-0022921,ARI,2018,L.Fitzgerald,112,6.115843,6.0,880.963379,734.0,72.544649,...,197.336049,6.291249,4.0,798.765869,835.0,73.160045,73.0,192.898148,192.5,190.784130
174,00-0022921,ARI,2019,L.Fitzgerald,107,6.291249,4.0,798.765869,835.0,73.160045,...,190.784130,2.179992,1.0,524.440125,409.0,52.608162,54.0,118.390869,100.9,118.132127
698,00-0022943,NO,2018,B.Watson,46,4.272406,2.0,365.639679,400.0,29.137302,...,91.335703,1.391622,0.0,176.140900,173.0,15.453680,17.0,42.463737,34.3,41.417503
1408,00-0025394,WAS,2018,A.Peterson,26,0.421688,1.0,143.061844,208.0,20.414077,...,37.250391,0.317519,0.0,109.799454,142.0,17.924322,17.0,31.677334,31.2,30.809381
1058,00-0025396,NO,2018,T.Ginn,30,2.348390,2.0,283.107178,209.0,17.385297,...,59.786354,4.093087,2.0,499.710510,421.0,32.226284,30.0,107.252998,84.1,106.755857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,00-0038117,NYG,2022,W.Robinson,31,0.678511,1.0,224.932175,227.0,22.078208,...,48.642494,0.979885,1.0,527.911743,525.0,56.055348,60.0,117.087814,118.5,114.725832
1220,00-0038120,NYJ,2022,Br.Hall,31,1.125437,1.0,192.434555,218.0,21.799425,...,47.795503,1.147388,4.0,498.789948,591.0,73.959531,76.0,131.883835,159.1,130.722856
435,00-0038124,GB,2022,C.Watson,66,5.528022,7.0,560.143677,611.0,38.034695,...,127.217193,6.063076,5.0,439.387939,422.0,28.629502,28.0,108.546494,100.2,108.946753
510,00-0038129,TB,2022,C.Otton,65,4.300029,2.0,437.800903,391.0,45.103641,...,114.683905,3.328940,4.0,453.356232,452.0,46.431071,47.0,112.547104,116.2,111.740336


In [46]:
df_lagged.tail(11)

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,xFPs_next,fantasy_points_next,compositeXFP_next
521,00-0037816,GB,2022,R.Doubs,67,2.434815,3.0,541.177368,425.0,44.964937,...,113.691567,6.992570,8.0,774.804138,674.0,57.829159,59.0,176.095627,174.4,177.264998
607,00-0037838,BAL,2022,I.Likely,60,3.109599,3.0,429.492950,373.0,40.330922,...,101.937810,1.534243,5.0,284.859711,411.0,27.930377,30.0,66.320175,101.1,65.621805
1445,00-0038041,DAL,2022,J.Ferguson,22,0.830617,2.0,140.835526,174.0,16.795957,...,35.863211,6.797571,5.0,647.428467,761.0,71.300789,71.0,178.171982,177.1,176.829060
1159,00-0038090,KC,2022,S.Moore,33,0.578630,0.0,265.516174,250.0,23.407232,...,53.430629,1.910576,1.0,288.433990,244.0,24.452009,21.0,66.717201,51.4,64.758867
757,00-0038104,NE,2022,T.Thornton,45,3.185590,2.0,401.870117,247.0,26.600075,...,85.900623,0.586035,0.0,184.883804,91.0,15.103351,13.0,38.596474,22.1,37.107941
1168,00-0038115,NYG,2022,D.Bellinger,35,0.768170,2.0,223.158844,268.0,24.952157,...,51.877062,0.119813,0.0,197.065216,255.0,21.681777,25.0,42.381401,50.5,42.107177
1208,00-0038117,NYG,2022,W.Robinson,31,0.678511,1.0,224.932175,227.0,22.078208,...,48.642494,0.979885,1.0,527.911743,525.0,56.055348,60.0,117.087814,118.5,114.725832
1220,00-0038120,NYJ,2022,Br.Hall,31,1.125437,1.0,192.434555,218.0,21.799425,...,47.795503,1.147388,4.0,498.789948,591.0,73.959531,76.0,131.883835,159.1,130.722856
435,00-0038124,GB,2022,C.Watson,66,5.528022,7.0,560.143677,611.0,38.034695,...,127.217193,6.063076,5.0,439.387939,422.0,28.629502,28.0,108.546494,100.2,108.946753
510,00-0038129,TB,2022,C.Otton,65,4.300029,2.0,437.800903,391.0,45.103641,...,114.683905,3.328940,4.0,453.356232,452.0,46.431071,47.0,112.547104,116.2,111.740336


In [47]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass', 'xFPs', 'fantasy_points', 'compositeXFP']:
    correlation = df_lagged[col].corr(df_lagged[f'{col}_next'])
    #print(df_lagged)
    correlations[col] = correlation

# Display the results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation with Next Season'])
print(correlations_df)

                Correlation with Next Season
xTDs                                0.639841
pass_touchdown                      0.487664
xYards                              0.712754
yards_gained                        0.687642
cp                                  0.611072
complete_pass                       0.620011
xFPs                                0.678747
fantasy_points                      0.662647
compositeXFP                        0.678789


In [48]:
correlations_df.drop(['xFPs'],inplace=True)

In [68]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for metric in ['pass_touchdown', 'yards_gained', 'complete_pass','fantasy_points']:
    if metric == 'pass_touchdown':
        predictor = 'xTDs'
    if metric == 'yards_gained':
        predictor = 'xYards'
    if metric == 'complete_pass':
        predictor = 'cp'
    if metric == 'fantasy_points':
        predictor = 'compositeXFP'
    #print(f"{predictor} to predict {metric}")
    correlation = df_lagged[predictor].corr(df_lagged[f'{metric}_next'])
    correlations[metric] = correlation

# Display the results
x_stat_correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Expected Stat Correlation with Actual Next Season Target'])
print(x_stat_correlations_df)

                Expected Stat Correlation with Actual Next Season Target
pass_touchdown                                           0.519842       
yards_gained                                             0.679533       
complete_pass                                            0.604711       
fantasy_points                                           0.648107       


In [50]:
correlations_df.merge(x_stat_correlations_df,right_index=True,left_index=True,how='outer').sort_values('Correlation with Next Season',ascending=False)

,Correlation with Next Season,Expected Stat Correlation with Actual Next Season Target
xYards,0.712754,NaN
yards_gained,0.687642,0.679533
compositeXFP,0.678789,NaN
fantasy_points,0.662647,0.648107
xTDs,0.639841,NaN
complete_pass,0.620011,0.604711
cp,0.611072,NaN
pass_touchdown,0.487664,0.519842


In [51]:
df_lagged[['xTDs','pass_touchdown','pass_touchdown_next']].corr()

,xTDs,pass_touchdown,pass_touchdown_next
xTDs,1.000000,0.823817,0.519842
pass_touchdown,0.823817,1.000000,0.487664
pass_touchdown_next,0.519842,0.487664,1.000000


In [52]:
df_lagged[['xYards','yards_gained','yards_gained_next']].corr()

,xYards,yards_gained,yards_gained_next
xYards,1.000000,0.962869,0.679533
yards_gained,0.962869,1.000000,0.687642
yards_gained_next,0.679533,0.687642,1.000000


In [53]:
df_lagged

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,xFPs_next,fantasy_points_next,compositeXFP_next
152,00-0022921,ARI,2018,L.Fitzgerald,112,6.115843,6.0,880.963379,734.0,72.544649,...,197.336049,6.291249,4.0,798.765869,835.0,73.160045,73.0,192.898148,192.5,190.784130
174,00-0022921,ARI,2019,L.Fitzgerald,107,6.291249,4.0,798.765869,835.0,73.160045,...,190.784130,2.179992,1.0,524.440125,409.0,52.608162,54.0,118.390869,100.9,118.132127
698,00-0022943,NO,2018,B.Watson,46,4.272406,2.0,365.639679,400.0,29.137302,...,91.335703,1.391622,0.0,176.140900,173.0,15.453680,17.0,42.463737,34.3,41.417503
1408,00-0025394,WAS,2018,A.Peterson,26,0.421688,1.0,143.061844,208.0,20.414077,...,37.250391,0.317519,0.0,109.799454,142.0,17.924322,17.0,31.677334,31.2,30.809381
1058,00-0025396,NO,2018,T.Ginn,30,2.348390,2.0,283.107178,209.0,17.385297,...,59.786354,4.093087,2.0,499.710510,421.0,32.226284,30.0,107.252998,84.1,106.755857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,00-0038117,NYG,2022,W.Robinson,31,0.678511,1.0,224.932175,227.0,22.078208,...,48.642494,0.979885,1.0,527.911743,525.0,56.055348,60.0,117.087814,118.5,114.725832
1220,00-0038120,NYJ,2022,Br.Hall,31,1.125437,1.0,192.434555,218.0,21.799425,...,47.795503,1.147388,4.0,498.789948,591.0,73.959531,76.0,131.883835,159.1,130.722856
435,00-0038124,GB,2022,C.Watson,66,5.528022,7.0,560.143677,611.0,38.034695,...,127.217193,6.063076,5.0,439.387939,422.0,28.629502,28.0,108.546494,100.2,108.946753
510,00-0038129,TB,2022,C.Otton,65,4.300029,2.0,437.800903,391.0,45.103641,...,114.683905,3.328940,4.0,453.356232,452.0,46.431071,47.0,112.547104,116.2,111.740336


# Intra-season Comparison

In [54]:
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [55]:
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'season', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

test_df = df[new_predictors]

test_df.head()

,air_yards,yardline_100,ydstogo,down,pass_location,season,qb_hit,end_zone_target,distance_to_EZ_after_target
2,8.0,80.0,15,1.0,right,2018,0.0,False,72.0
5,4.0,39.0,10,1.0,right,2018,0.0,False,35.0
6,-3.0,39.0,10,2.0,left,2018,0.0,False,42.0
7,24.0,39.0,10,3.0,left,2018,0.0,False,15.0
10,1.0,1.0,1,3.0,right,2018,0.0,True,0.0


In [56]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [57]:
test_df = pd.get_dummies(test_df, columns=['pass_location'], drop_first=True)


# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict(test_df)
df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['cp'] * ppr + df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_19252\2134994946.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\2134994946.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_19252\2134994946.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [58]:

# Filter weeks 1-9 and weeks 10-18
first_half = df[df['week'].between(1, 9)]
second_half = df[df['week'].between(10, 18)]

# Define metrics to aggregate
metrics = ['pass','xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass', 'xFPs', 'fantasy_points', 'compositeXFP']

# Aggregate metrics for each player-season for weeks 1-9
first_half_agg = first_half.groupby(['receiver_player_id', 'season'])[metrics].sum()
first_half_agg = first_half_agg.add_suffix('_wk1_9')

# Aggregate metrics for each player-season for weeks 10-18
second_half_agg = second_half.groupby(['receiver_player_id', 'season'])[metrics].sum()
second_half_agg = second_half_agg.add_suffix('_wk10_18')

# Merge the two aggregated DataFrames
df_aggregated = pd.merge(first_half_agg, second_half_agg, left_index=True, right_index=True)


In [59]:
df_aggregated = df_aggregated[(df_aggregated['pass_wk1_9'] >= 5) & (df_aggregated['pass_wk10_18'] >= 5)]


In [60]:
# Create a dictionary to store correlations for each metric
correlations = {}

# Calculate correlation for each metric between weeks 1-9 and weeks 10-18
for metric in metrics:
    correlation = df_aggregated[f'{metric}_wk1_9'].corr(df_aggregated[f'{metric}_wk10_18'])
    correlations[metric] = correlation

# Display the correlation results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation between Weeks 1-9 and 10-18'])
print(correlations_df)


                Correlation between Weeks 1-9 and 10-18
pass                                           0.712226
xTDs                                           0.579130
pass_touchdown                                 0.420054
xYards                                         0.751190
yards_gained                                   0.711520
cp                                             0.687478
complete_pass                                  0.686201
xFPs                                           0.721933
fantasy_points                                 0.688007
compositeXFP                                   0.721844


In [70]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18
fantasy_points_wk1_9,1.000000,0.956955,0.688007
compositeXFP_wk1_9,0.956955,1.000000,0.687332
fantasy_points_wk10_18,0.688007,0.687332,1.000000


In [62]:
df_aggregated[['yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18']].corr()

,yards_gained_wk1_9,xYards_wk1_9,yards_gained_wk10_18
yards_gained_wk1_9,1.000000,0.952486,0.711520
xYards_wk1_9,0.952486,1.000000,0.705179
yards_gained_wk10_18,0.711520,0.705179,1.000000


In [63]:
df_aggregated[['pass_touchdown_wk1_9','xTDs_wk1_9','pass_touchdown_wk10_18']].corr()

,pass_touchdown_wk1_9,xTDs_wk1_9,pass_touchdown_wk10_18
pass_touchdown_wk1_9,1.000000,0.782478,0.420054
xTDs_wk1_9,0.782478,1.000000,0.461352
pass_touchdown_wk10_18,0.420054,0.461352,1.000000
